# 毕业论文`GRAD`
## 建立多元线性回归模型，定量评估影响

*`Evan`*\
*`2023-11-16`*
---

In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *
from RandomForest import read_data, rf_importance

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

## 读取数据：将三年数据拼接

In [2]:
lowyears  = [2014,2015,2016]
highyears = [2019,2021,2022]
month     = 'Sep'
region    = 'PRD'
datapath  = datadir + 'Contribution/data/'

df_low  = read_data(lowyears,month,region,datapath)
df_high = read_data(highyears,month,region,datapath)


In [3]:
df_low[df_low['SOL_RAD']<100].index.hour.value_counts()

0     90
1     90
2     90
3     90
4     90
5     90
6     90
7     90
19    90
20    90
21    90
22    90
23    90
18    44
dtype: int64

# 保留白天时刻，去除夜间时刻

In [4]:
day_low  = df_low.between_time('8:00','17:00')
day_high = df_high.between_time('8:00','17:00')

## 建立模型

In [5]:
# 设置变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',]
            # 'NO2', 'VOC', 'PM25', 'ISOP'] # 'PBLH',
target = 'O3'

low_importance = rf_importance(day_low,variants,target)
high_importance = rf_importance(day_high,variants,target)

In [6]:
day_low = day_low.reset_index()
day_low.drop(columns=['index'],inplace=True)
day_high = day_high.reset_index()
day_high.drop(columns=['index'],inplace=True)

day_diff = day_high - day_low

diff_importance = rf_importance(day_diff,variants,target)


## 输出结果

In [7]:
# 合并两组结果，对列重命名
dfoutput = pd.concat([low_importance,high_importance,diff_importance],axis=1)
dfoutput.columns=['Low','High','Diff']

# randomforest_concatinate_PRDaveraged
filepath = 'D:/data/Graduation/Contribution/RandomForest_output/'
filename = f'RF_{region}_day.xlsx'
dfoutput.to_excel(filepath + filename,index=True)

# 保留夜间时刻，去除白天时刻

In [8]:
night_low  = df_low.between_time('18:00','7:00')
night_high = df_high.between_time('18:00','7:00')

In [12]:
# 设置变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',]
            # 'NO2', 'VOC', 'PM25', 'ISOP'] # 'PBLH',
target = 'O3'

low_importance = rf_importance(night_low,variants,target)
high_importance = rf_importance(night_high,variants,target)


In [13]:
night_low = night_low.reset_index()
night_low.drop(columns=['index'],inplace=True)
night_high = night_high.reset_index()
night_high.drop(columns=['index'],inplace=True)

night_diff = night_high - night_low

diff_importance = rf_importance(night_diff,variants,target)


In [14]:
# 合并两组结果，对列重命名
dfoutput = pd.concat([low_importance,high_importance,diff_importance],axis=1)
dfoutput.columns=['Low','High','Diff']

# randomforest_concatinate_PRDaveraged
filepath = 'D:/data/Graduation/Contribution/RandomForest_output/'
filename = f'RF_{region}_night.xlsx'
dfoutput.to_excel(filepath + filename,index=True)